In [63]:
import torch
from models import *
from utils import *

Load in the teacher models (and train the independent student model)

In [64]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg16_bn", pretrained=True)
device = 'cuda'if torch.cuda.is_available() else 'cpu'
print(eval(model.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Using cache found in C:\Users\PCF/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


(2.092914149260065, array(0.6892, dtype=float32))


In [65]:
teacher_model_vgg16_bn = vgg16_bn_f(True,100,model.state_dict())
def eval_f(model,test_loader,loss_func,device):
    total_loss = 0
    pred_list = None
    for i,batch in enumerate(test_loader):
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        pred,_ = model(x_batch)
        loss = loss_func(pred,y_batch.reshape(-1).long())
        pred_list = torch.cat([pred_list,torch.argmax(pred,dim=1) == y_batch]) if pred_list is not None else torch.argmax(pred,dim=1) == y_batch

        total_loss += loss.item()

    total_loss /= len(test_loader)

    return total_loss,pred_list.float().mean().cpu().numpy()
print(eval_f(teacher_model_vgg16_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Loading pretrained weights
Mapping: block0.0.weight -> features.0.weight
Mapping: block0.1.weight -> features.1.weight
Mapping: block0.3.weight -> features.3.weight
Mapping: block0.4.weight -> features.4.weight
Mapping: block1.0.weight -> features.7.weight
Mapping: block1.1.weight -> features.8.weight
Mapping: block1.3.weight -> features.10.weight
Mapping: block1.4.weight -> features.11.weight
Mapping: block2.0.weight -> features.14.weight
Mapping: block2.1.weight -> features.15.weight
Mapping: block2.3.weight -> features.17.weight
Mapping: block2.4.weight -> features.18.weight
Mapping: block2.6.weight -> features.20.weight
Mapping: block2.7.weight -> features.21.weight
Mapping: block3.0.weight -> features.24.weight
Mapping: block3.1.weight -> features.25.weight
Mapping: block3.3.weight -> features.27.weight
Mapping: block3.4.weight -> features.28.weight
Mapping: block3.6.weight -> features.30.weight
Mapping: block3.7.weight -> features.31.weight
Mapping: block4.0.weight -> features.34

In [69]:
for epoch in range(5):
    independent_train_f(teacher_model_vgg16_bn,train_loader,nn.CrossEntropyLoss(),torch.optim.Adam(teacher_model_vgg16_bn.parameters(),0.001),device)

In [70]:
eval_f(teacher_model_vgg16_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda')

(1.6784727193747357, array(0.6185, dtype=float32))

In [71]:
torch.save(teacher_model_vgg16_bn.state_dict(),'teacher_model_vgg16_bn.pth')

In [41]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg19_bn", pretrained=True)
device = 'cuda'if torch.cuda.is_available() else 'cpu'
print(eval(model.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Using cache found in C:\Users\PCF/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


(2.2601462093887816, array(0.69409996, dtype=float32))


In [42]:
teacher_model_vgg19_bn = vgg19_bn_f(True,100,model.state_dict())
print(eval_f(teacher_model_vgg19_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Loading pretrained weights
Mapping: block0.0.weight -> features.0.weight
Mapping: block0.1.weight -> features.1.weight
Mapping: block0.3.weight -> features.3.weight
Mapping: block0.4.weight -> features.4.weight
Mapping: block1.0.weight -> features.7.weight
Mapping: block1.1.weight -> features.8.weight
Mapping: block1.3.weight -> features.10.weight
Mapping: block1.4.weight -> features.11.weight
Mapping: block2.0.weight -> features.14.weight
Mapping: block2.1.weight -> features.15.weight
Mapping: block2.3.weight -> features.17.weight
Mapping: block2.4.weight -> features.18.weight
Mapping: block2.6.weight -> features.20.weight
Mapping: block2.7.weight -> features.21.weight
Mapping: block2.9.weight -> features.23.weight
Mapping: block2.10.weight -> features.24.weight
Mapping: block3.0.weight -> features.27.weight
Mapping: block3.1.weight -> features.28.weight
Mapping: block3.3.weight -> features.30.weight
Mapping: block3.4.weight -> features.31.weight
Mapping: block3.6.weight -> features.3

In [60]:
for epoch in range(5):
    independent_train_f(teacher_model_vgg19_bn,train_loader,nn.CrossEntropyLoss(),torch.optim.Adam(teacher_model_vgg16_bn.parameters(),0.001),device)

In [62]:
eval_f(teacher_model_vgg19_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda')

(2.7729790317025156, array(0.4992, dtype=float32))

In [59]:
torch.save(teacher_model_vgg19_bn.state_dict(),'teacher_model_vgg19_bn.pth')

In [91]:
EPOCHS = 30

In [92]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
student_model = vgg11_bn_f(False,100)
for param in student_model.parameters():
    param.requires_grad = True

In [93]:
optimizer = torch.optim.Adam(student_model.parameters(),0.002)
loss_func = torch.nn.CrossEntropyLoss()

In [94]:
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = independent_train_f(student_model,train_loader,loss_func,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))

Epoch: 1 Train Loss: 4.38 Train Accuracy: 0.02
Epoch: 2 Train Loss: 4.20 Train Accuracy: 0.04
Epoch: 3 Train Loss: 4.09 Train Accuracy: 0.05
Epoch: 4 Train Loss: 4.00 Train Accuracy: 0.06
Epoch: 5 Train Loss: 3.91 Train Accuracy: 0.06
Epoch: 6 Train Loss: 3.81 Train Accuracy: 0.08
Epoch: 7 Train Loss: 3.72 Train Accuracy: 0.09
Epoch: 8 Train Loss: 3.63 Train Accuracy: 0.10
Epoch: 9 Train Loss: 3.56 Train Accuracy: 0.11
Epoch: 10 Train Loss: 3.48 Train Accuracy: 0.12
Epoch: 11 Train Loss: 3.43 Train Accuracy: 0.13
Epoch: 12 Train Loss: 3.35 Train Accuracy: 0.15
Epoch: 13 Train Loss: 3.27 Train Accuracy: 0.15
Epoch: 14 Train Loss: 3.21 Train Accuracy: 0.17
Epoch: 15 Train Loss: 3.14 Train Accuracy: 0.18
Epoch: 16 Train Loss: 3.09 Train Accuracy: 0.19
Epoch: 17 Train Loss: 3.02 Train Accuracy: 0.20
Epoch: 18 Train Loss: 2.95 Train Accuracy: 0.22
Epoch: 19 Train Loss: 2.90 Train Accuracy: 0.23
Epoch: 20 Train Loss: 2.84 Train Accuracy: 0.24
Epoch: 21 Train Loss: 2.79 Train Accuracy: 0.25
E

In [95]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.352
Final Test Accuracy: 0.3204
Final Train Loss: 2.3490027318830076
Final Test Loss: 2.599426288513621


In [96]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_student_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_student_ts.csv")

In [97]:
torch.save(student_model.state_dict(),"vgg11_cifar100_student.pth")

#### 3.1.2.1: Basic Logit Matching

In [79]:
from tqdm import tqdm
student_model = vgg11_bn_f(False,100)
teacher_model_vgg16_bn = vgg16_bn_f(False,100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_model_vgg16_bn.pth")) 
for param in student_model.parameters():
    param.requires_grad = True          
for param in teacher_model_vgg16_bn.parameters():
    param.requires_grad = False
teacher_model = teacher_model_vgg16_bn.to(device)
student_model = student_model.to(device)
optimizer = torch.optim.Adam(student_model.parameters(),lr=0.002)
loss_func = torch.nn.CrossEntropyLoss()
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = train(student_model,teacher_model,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


C:\Users\PCF\AppData\Local\Temp\ipykernel_25804\3696582385.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_mod

Epoch: 1 Train Loss: 7.67 Train Accuracy: 0.04
Epoch: 2 Train Loss: 7.10 Train Accuracy: 0.06
Epoch: 3 Train Loss: 6.63 Train Accuracy: 0.08
Epoch: 4 Train Loss: 6.22 Train Accuracy: 0.11
Epoch: 5 Train Loss: 5.85 Train Accuracy: 0.14
Epoch: 6 Train Loss: 5.52 Train Accuracy: 0.16
Epoch: 7 Train Loss: 5.26 Train Accuracy: 0.19
Epoch: 8 Train Loss: 5.04 Train Accuracy: 0.21
Epoch: 9 Train Loss: 4.82 Train Accuracy: 0.23
Epoch: 10 Train Loss: 4.61 Train Accuracy: 0.25
Epoch: 11 Train Loss: 4.45 Train Accuracy: 0.27
Epoch: 12 Train Loss: 4.29 Train Accuracy: 0.29
Epoch: 13 Train Loss: 4.12 Train Accuracy: 0.30
Epoch: 14 Train Loss: 4.01 Train Accuracy: 0.32
Epoch: 15 Train Loss: 3.88 Train Accuracy: 0.34
Epoch: 16 Train Loss: 3.76 Train Accuracy: 0.35
Epoch: 17 Train Loss: 3.63 Train Accuracy: 0.37
Epoch: 18 Train Loss: 3.53 Train Accuracy: 0.38
Epoch: 19 Train Loss: 3.42 Train Accuracy: 0.40
Epoch: 20 Train Loss: 3.33 Train Accuracy: 0.41
Epoch: 21 Train Loss: 3.23 Train Accuracy: 0.42
E

In [80]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.52638
Final Test Accuracy: 0.4799
Final Train Loss: 2.600411142839495
Final Test Loss: 2.1949146340607077


In [81]:
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_blm_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_blm_ts.csv")

In [82]:
torch.save(student_model.state_dict(),"vgg11_cifar100_blm.pth")

#### 3.1.2.2: Label Smooth Regularization

In [83]:
from tqdm import tqdm

student_model = vgg11_bn_f(False,100)
teacher_model_vgg16_bn = vgg16_bn_f(False,100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_model_vgg16_bn.pth")) 
for param in student_model.parameters():
    param.requires_grad = True          
for param in teacher_model_vgg16_bn.parameters():
    param.requires_grad = False
teacher_model = teacher_model_vgg16_bn.to(device)
student_model = student_model.to(device)
optimizer = torch.optim.Adam(student_model.parameters(),lr=0.002)

for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = train(student_model,teacher_model,train_loader,optimizer,device,types="lsr")
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))

C:\Users\PCF\AppData\Local\Temp\ipykernel_25804\2179157929.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_mod

Epoch: 1 Train Loss: 5.34 Train Accuracy: 0.03
Epoch: 2 Train Loss: 5.06 Train Accuracy: 0.05
Epoch: 3 Train Loss: 4.84 Train Accuracy: 0.07
Epoch: 4 Train Loss: 4.65 Train Accuracy: 0.09
Epoch: 5 Train Loss: 4.44 Train Accuracy: 0.11
Epoch: 6 Train Loss: 4.27 Train Accuracy: 0.13
Epoch: 7 Train Loss: 4.10 Train Accuracy: 0.15
Epoch: 8 Train Loss: 3.95 Train Accuracy: 0.17
Epoch: 9 Train Loss: 3.81 Train Accuracy: 0.20
Epoch: 10 Train Loss: 3.68 Train Accuracy: 0.22
Epoch: 11 Train Loss: 3.54 Train Accuracy: 0.24
Epoch: 12 Train Loss: 3.44 Train Accuracy: 0.26
Epoch: 13 Train Loss: 3.35 Train Accuracy: 0.27
Epoch: 14 Train Loss: 3.24 Train Accuracy: 0.30
Epoch: 15 Train Loss: 3.15 Train Accuracy: 0.31
Epoch: 16 Train Loss: 3.05 Train Accuracy: 0.33
Epoch: 17 Train Loss: 2.98 Train Accuracy: 0.35
Epoch: 18 Train Loss: 2.90 Train Accuracy: 0.36
Epoch: 19 Train Loss: 2.83 Train Accuracy: 0.38
Epoch: 20 Train Loss: 2.75 Train Accuracy: 0.39
Epoch: 21 Train Loss: 2.68 Train Accuracy: 0.41
E

In [84]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.52384
Final Test Accuracy: 0.4676
Final Train Loss: 2.191634896618631
Final Test Loss: 2.1344546998382374


In [85]:
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_lsr_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_lsr_ts.csv")

In [86]:
torch.save(student_model.state_dict(),"vgg11_cifar100_lsr.pth")

#### 3.2.1.3: Decoupled Knowledge Distillation

In [87]:
from tqdm import tqdm

student_model = vgg11_bn_f(False,100)
teacher_model_vgg16_bn = vgg16_bn_f(False,100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_model_vgg16_bn.pth")) 
for param in student_model.parameters():
    param.requires_grad = True          
for param in teacher_model_vgg16_bn.parameters():
    param.requires_grad = False

tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}
teacher_model = teacher_model_vgg16_bn.to(device)
student_model = student_model.to(device)
optimizer = torch.optim.Adam(student_model.parameters(),lr=0.002)
loss_func = torch.nn.CrossEntropyLoss()
for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = train(student_model,teacher_model,train_loader,optimizer,device,types="dkd")
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))

C:\Users\PCF\AppData\Local\Temp\ipykernel_25804\3597643403.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_mod

Epoch: 1 Train Loss: 9.87 Train Accuracy: 0.04
Epoch: 2 Train Loss: 9.06 Train Accuracy: 0.07
Epoch: 3 Train Loss: 8.36 Train Accuracy: 0.10
Epoch: 4 Train Loss: 7.72 Train Accuracy: 0.13
Epoch: 5 Train Loss: 7.23 Train Accuracy: 0.16
Epoch: 6 Train Loss: 6.83 Train Accuracy: 0.19
Epoch: 7 Train Loss: 6.48 Train Accuracy: 0.21
Epoch: 8 Train Loss: 6.15 Train Accuracy: 0.24
Epoch: 9 Train Loss: 5.88 Train Accuracy: 0.27
Epoch: 10 Train Loss: 5.62 Train Accuracy: 0.29
Epoch: 11 Train Loss: 5.39 Train Accuracy: 0.31
Epoch: 12 Train Loss: 5.17 Train Accuracy: 0.33
Epoch: 13 Train Loss: 5.02 Train Accuracy: 0.35
Epoch: 14 Train Loss: 4.83 Train Accuracy: 0.37
Epoch: 15 Train Loss: 4.68 Train Accuracy: 0.39
Epoch: 16 Train Loss: 4.53 Train Accuracy: 0.41
Epoch: 17 Train Loss: 4.43 Train Accuracy: 0.42
Epoch: 18 Train Loss: 4.29 Train Accuracy: 0.43
Epoch: 19 Train Loss: 4.20 Train Accuracy: 0.44
Epoch: 20 Train Loss: 4.11 Train Accuracy: 0.46
Epoch: 21 Train Loss: 3.96 Train Accuracy: 0.47
E

In [88]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.55597997
Final Test Accuracy: 0.4978
Final Train Loss: 3.360415622096537
Final Test Loss: 2.030430110396853


In [89]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_dkd_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_dkd_ts.csv")

In [90]:
torch.save(student_model.state_dict(),"vgg11_cifar100_dkd.pth")